In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sqlite3
from sqlalchemy import create_engine

In [2]:
book = pd.read_csv(r"C:\Users\esait\Books_Data.csv")
book.columns = book.columns.str.replace(' ', '_')
book.head()

,index,Publishing_Year,Book_Name,Author,language_code,Author_Rating,Book_average_rating,Book_ratings_count,genre,gross_sales,publisher_revenue,sale_price,sales_rank,Publisher_,units_sold
0,0,1975.0,Beowulf,"Unknown, Seamus Heaney",en-US,Novice,3.42,155903,genre fiction,34160.0,20496.0,4.88,1,HarperCollins Publishers,7000
1,1,1987.0,Batman: Year One,"Frank Miller, David Mazzucchelli, Richmond Lew...",eng,Intermediate,4.23,145267,genre fiction,12437.5,7462.5,1.99,2,HarperCollins Publishers,6250
2,2,2015.0,Go Set a Watchman,Harper Lee,eng,Novice,3.31,138669,genre fiction,47795.0,28677.0,8.69,3,"Amazon Digital Services, Inc.",5500
3,3,2008.0,When You Are Engulfed in Flames,David Sedaris,en-US,Intermediate,4.04,150898,fiction,41250.0,24750.0,7.50,3,Hachette Book Group,5500
4,4,2011.0,Daughter of Smoke & Bone,Laini Taylor,eng,Intermediate,4.04,198283,genre fiction,37952.5,22771.5,7.99,4,Penguin Group (USA) LLC,4750


In [3]:
book.isnull().sum()

index                   0
Publishing_Year         1
Book_Name              23
Author                  0
language_code          53
Author_Rating           0
Book_average_rating     0
Book_ratings_count      0
genre                   0
gross_sales             0
publisher_revenue       0
sale_price              0
sales_rank              0
Publisher_              0
units_sold              0
dtype: int64

Create a database and fill a table with the .csv file data to proceed with SQL queries

In [4]:
book_db = sqlite3.connect('book_db.db')
cur = book_db.cursor()
book.to_sql('book_sales', book_db, if_exists='replace', index=False)
book_db.commit()

Create a function to update the original data table and create a new .CSV with these modifications

In [5]:
def update_data(query, db_path='book_db.db'):
    book_db = sqlite3.connect(db_path)
    cur = book_db.cursor()
    cur.execute(query)
    book_db.commit()
    book_update = pd.read_sql_query("SELECT * FROM book_sales", book_db)
    book_update.to_csv('updated_book.csv', index=False)
    book_db.close()

In [6]:
%%capture
%load_ext sql
%sql sqlite:///C:\Users\esait\book_db.db

In [7]:
%%sql
SELECT * FROM book_sales LIMIT 5;

 * sqlite:///C:\Users\esait\book_db.db
Done.


index,Publishing_Year,Book_Name,Author,language_code,Author_Rating,Book_average_rating,Book_ratings_count,genre,gross_sales,publisher_revenue,sale_price,sales_rank,Publisher_,units_sold
0,1975.0,Beowulf,"Unknown, Seamus Heaney",en-US,Novice,3.42,155903,genre fiction,34160.0,20496.0,4.88,1,HarperCollins Publishers,7000
1,1987.0,Batman: Year One,"Frank Miller, David Mazzucchelli, Richmond Lewis, Dennis O'Neil",eng,Intermediate,4.23,145267,genre fiction,12437.5,7462.5,1.99,2,HarperCollins Publishers,6250
2,2015.0,Go Set a Watchman,Harper Lee,eng,Novice,3.31,138669,genre fiction,47795.0,28677.0,8.69,3,"Amazon Digital Services, Inc.",5500
3,2008.0,When You Are Engulfed in Flames,David Sedaris,en-US,Intermediate,4.04,150898,fiction,41250.0,24750.0,7.5,3,Hachette Book Group,5500
4,2011.0,Daughter of Smoke & Bone,Laini Taylor,eng,Intermediate,4.04,198283,genre fiction,37952.5,22771.5,7.99,4,Penguin Group (USA) LLC,4750


In [8]:
%%sql
PRAGMA table_info(book_sales);

 * sqlite:///C:\Users\esait\book_db.db
Done.


cid,name,type,notnull,dflt_value,pk
0,index,INTEGER,0,None,0
1,Publishing_Year,REAL,0,None,0
2,Book_Name,TEXT,0,None,0
3,Author,TEXT,0,None,0
4,language_code,TEXT,0,None,0
5,Author_Rating,TEXT,0,None,0
6,Book_average_rating,REAL,0,None,0
7,Book_ratings_count,INTEGER,0,None,0
8,genre,TEXT,0,None,0
9,gross_sales,REAL,0,None,0


In [9]:
%%sql
ALTER TABLE book_sales 
RENAME COLUMN Publisher_ TO Publisher;

 * sqlite:///C:\Users\esait\book_db.db
Done.


1


In [10]:
%%sql
SELECT Publisher, SUM(units_sold) AS 'Units Sold', ROUND(SUM(gross_sales), 3) AS 'Total Gross Sales', ROUND(SUM(publisher_revenue), 3) AS 'Total Publisher Revenue'
FROM book_sales
GROUP BY 1
ORDER BY 3
DESC;

 * sqlite:///C:\Users\esait\book_db.db
Done.


Publisher,Units Sold,Total Gross Sales,Total Publisher Revenue
"Amazon Digital Services, Inc.",6074136,729809.01,148244.178
Penguin Group (USA) LLC,934303,356362.47,213817.482
Random House LLC,1315958,315975.37,189585.222
Hachette Book Group,543821,229790.83,137874.498
HarperCollins Publishers,563409,214130.43,128478.258
Simon and Schuster Digital Sales Inc,501928,78313.92,46988.352
Macmillan,334929,53927.06,32356.236
HarperCollins Publishing,44358,4718.01,2830.806
HarperCollins Christian Publishing,41527,3559.45,2135.67


I noticed a discrepancy between HarperCollins Publishers and HarperCollins Publishing. Inquired into whether this is a clerical error or if HarperCollins draws a corporate distinction between the two. Confirmed that the "Publishing" title is a clerical mistake. Update relevant entries. Also, HarperCollins Christian Publishing is a division of HarperCollins Publishers. For this analysis, given that the other publishers in the dataset are distinct parent companies, we will fold the HarperCollins Christian Publishing data into HarperCollins Publisher. I also changed the publishing year column type from text to integer.

In [11]:
%%sql
UPDATE book_sales
SET Publisher = 'HarperCollins Publishers'
WHERE Publisher = 'HarperCollins Publishing';
UPDATE book_sales
SET Publisher = 'HarperCollins Publishers'
WHERE Publisher = 'HarperCollins Christian Publishing';
UPDATE book_sales
SET Publishing_Year = CAST(Publishing_Year AS INT)
WHERE Publishing_Year IS NOT NULL AND Publishing_Year GLOB '[0-9]*';

 * sqlite:///C:\Users\esait\book_db.db
4 rows affected.
4 rows affected.
1063 rows affected.


[]

In [12]:
query = """UPDATE book_sales 
SET Publisher = 'HarperCollins Publishers' 
WHERE Publisher = 'HarperCollins Publishing';
"""
update_data(query)

In [13]:
query = """UPDATE book_sales 
SET Publisher = 'HarperCollins Publishers' 
WHERE Publisher = 'HarperCollins Christian Publishing';
"""
update_data(query)

In [14]:
query = """UPDATE book_sales 
SET Publishing_Year = CAST(Publishing_Year AS INT) 
WHERE Publishing_Year IS NOT NULL AND Publishing_Year GLOB '[0-9]*';
"""
update_data(query)

In [15]:
%%sql
SELECT DISTINCT Publisher FROM book_sales;

 * sqlite:///C:\Users\esait\book_db.db
Done.


Publisher
HarperCollins Publishers
"Amazon Digital Services, Inc."
Hachette Book Group
Penguin Group (USA) LLC
Random House LLC
Simon and Schuster Digital Sales Inc
Macmillan


In the genre column, I noted that there are duplicative entries ('genre fiction' and 'fiction'). For clarity and uniformity, I updated the relevant rows of the dataset to the single designation of 'fiction'.

In [16]:
%%sql
SELECT DISTINCT genre FROM book_sales;

 * sqlite:///C:\Users\esait\book_db.db
Done.


genre
genre fiction
fiction
nonfiction
children


In [17]:
%%sql
UPDATE book_sales
SET genre = 'fiction'
WHERE genre = 'genre fiction';

 * sqlite:///C:\Users\esait\book_db.db
822 rows affected.


[]

In [18]:
query = """UPDATE book_sales
SET genre = 'fiction'
WHERE genre = 'genre fiction';
"""
update_data(query)

In [19]:
book_upd = pd.read_csv(r"C:\Users\esait\updated_book.csv")
book_upd.head()

,index,Publishing_Year,Book_Name,Author,language_code,Author_Rating,Book_average_rating,Book_ratings_count,genre,gross_sales,publisher_revenue,sale_price,sales_rank,Publisher,units_sold
0,0,1975.0,Beowulf,"Unknown, Seamus Heaney",en-US,Novice,3.42,155903,fiction,34160.0,20496.0,4.88,1,HarperCollins Publishers,7000
1,1,1987.0,Batman: Year One,"Frank Miller, David Mazzucchelli, Richmond Lew...",eng,Intermediate,4.23,145267,fiction,12437.5,7462.5,1.99,2,HarperCollins Publishers,6250
2,2,2015.0,Go Set a Watchman,Harper Lee,eng,Novice,3.31,138669,fiction,47795.0,28677.0,8.69,3,"Amazon Digital Services, Inc.",5500
3,3,2008.0,When You Are Engulfed in Flames,David Sedaris,en-US,Intermediate,4.04,150898,fiction,41250.0,24750.0,7.50,3,Hachette Book Group,5500
4,4,2011.0,Daughter of Smoke & Bone,Laini Taylor,eng,Intermediate,4.04,198283,fiction,37952.5,22771.5,7.99,4,Penguin Group (USA) LLC,4750


In [20]:
%%sql
SELECT DISTINCT genre FROM book_sales;

 * sqlite:///C:\Users\esait\book_db.db
Done.


genre
fiction
nonfiction
children


In [21]:
%%sql
SELECT * FROM book_sales LIMIT 5;

 * sqlite:///C:\Users\esait\book_db.db
Done.


index,Publishing_Year,Book_Name,Author,language_code,Author_Rating,Book_average_rating,Book_ratings_count,genre,gross_sales,publisher_revenue,sale_price,sales_rank,Publisher,units_sold
0,1975.0,Beowulf,"Unknown, Seamus Heaney",en-US,Novice,3.42,155903,fiction,34160.0,20496.0,4.88,1,HarperCollins Publishers,7000
1,1987.0,Batman: Year One,"Frank Miller, David Mazzucchelli, Richmond Lewis, Dennis O'Neil",eng,Intermediate,4.23,145267,fiction,12437.5,7462.5,1.99,2,HarperCollins Publishers,6250
2,2015.0,Go Set a Watchman,Harper Lee,eng,Novice,3.31,138669,fiction,47795.0,28677.0,8.69,3,"Amazon Digital Services, Inc.",5500
3,2008.0,When You Are Engulfed in Flames,David Sedaris,en-US,Intermediate,4.04,150898,fiction,41250.0,24750.0,7.5,3,Hachette Book Group,5500
4,2011.0,Daughter of Smoke & Bone,Laini Taylor,eng,Intermediate,4.04,198283,fiction,37952.5,22771.5,7.99,4,Penguin Group (USA) LLC,4750


Publishers ordered by top publisher revenue and detailing units sold and gross sales

In [22]:
%%sql
SELECT Publisher, SUM(units_sold) AS 'Units Sold', ROUND(SUM(gross_sales), 2) AS 'Total Gross Sales', ROUND(SUM(publisher_revenue), 2) AS 'Total Publisher Revenue'
FROM book_sales
GROUP BY 1
ORDER BY 4
DESC;

 * sqlite:///C:\Users\esait\book_db.db
Done.


Publisher,Units Sold,Total Gross Sales,Total Publisher Revenue
Penguin Group (USA) LLC,934303,356362.47,213817.48
Random House LLC,1315958,315975.37,189585.22
"Amazon Digital Services, Inc.",6074136,729809.01,148244.18
Hachette Book Group,543821,229790.83,137874.5
HarperCollins Publishers,649294,222407.89,133444.73
Simon and Schuster Digital Sales Inc,501928,78313.92,46988.35
Macmillan,334929,53927.06,32356.24


Top 10 authors ordered by publisher revenue and detailing publisher, units sold and gross sales

In [23]:
%%sql
SELECT author, SUM(units_sold), ROUND(SUM(gross_sales),2) AS 'Total Gross Sales', 
ROUND(SUM(publisher_revenue), 2) AS 'Total Publisher Revenue', Publisher
FROM book_sales
GROUP BY 1
ORDER BY 4
DESC
LIMIT 10;

 * sqlite:///C:\Users\esait\book_db.db
Done.


Author,SUM(units_sold),Total Gross Sales,Total Publisher Revenue,Publisher
Harper Lee,5500,47795.0,28677.0,"Amazon Digital Services, Inc."
David Sedaris,15193,42323.41,25037.04,Hachette Book Group
Laini Taylor,8990,38278.41,22967.05,Penguin Group (USA) LLC
Charlaine Harris,47249,39453.08,22246.25,"Amazon Digital Services, Inc."
"Unknown, Seamus Heaney",7000,34160.0,20496.0,HarperCollins Publishers
Colleen Hoover,123522,30882.07,17877.62,HarperCollins Publishers
C.S. Lewis,8381,28972.57,17383.54,Penguin Group (USA) LLC
Jeffrey Eugenides,8163,27984.38,16790.63,Penguin Group (USA) LLC
Stephen King,278322,43322.65,16744.55,"Amazon Digital Services, Inc."
Charles Duhigg,3933,27491.67,16495.0,HarperCollins Publishers


Top 10 Penguin Group books ordered by publisher revenue and detailing units sold and gross sales

In [24]:
%%sql
SELECT book_name, author, Publishing_Year, SUM(units_sold) AS 'Units Sold', ROUND(SUM(gross_sales), 2) AS 'Total Gross Sales', 
ROUND(SUM(publisher_revenue), 2) AS 'Total Publisher Revenue', Book_average_rating
FROM book_sales
WHERE Publisher = 'Penguin Group (USA) LLC' AND Book_Name != 'None'
GROUP BY 1
ORDER BY 6
DESC
LIMIT 10;

 * sqlite:///C:\Users\esait\book_db.db
Done.


Book_Name,Author,Publishing_Year,Units Sold,Total Gross Sales,Total Publisher Revenue,Book_average_rating
Daughter of Smoke & Bone,Laini Taylor,2011.0,4750,37952.5,22771.5,4.04
The Virgin Suicides,Jeffrey Eugenides,1993.0,2466,26904.06,16142.44,3.83
The horse and his boy,C.S. Lewis,1954.0,3600,21564.0,12938.4,3.9
The Bourne Supremacy,Robert Ludlum,1986.0,1630,17783.3,10669.98,4.09
The Restaurant at the End of the Universe,Douglas Adams,1980.0,1446,11553.54,6932.12,4.22
Are You My Mother?,P.D. Eastman,1960.0,1769,10596.31,6357.79,4.16
The Wind in the Willows,"Kenneth Grahame, Gillian Avery",1908.0,305,10327.3,6196.38,3.98
Sofies Verden,"Jostein Gaarder, Paulette MÃ¸ller",1991.0,960,10224.0,6134.4,3.88
The Tale of Peter Rabbit,Beatrix Potter,1901.0,1415,9890.85,5934.51,4.17
Along for the Ride,Sarah Dessen,2009.0,785,9412.15,5647.29,4.08


Total publisher revenue and units sold by year, top 15 years

In [25]:
%%sql
SELECT CAST(Publishing_Year AS INT) AS 'Year Published', SUM(units_sold) AS 'Units Sold', ROUND(SUM(publisher_revenue), 2) AS 'Total Publisher Revenue'
FROM book_sales
GROUP BY 1
ORDER BY 3
DESC
LIMIT 15;

 * sqlite:///C:\Users\esait\book_db.db
Done.


Year Published,Units Sold,Total Publisher Revenue
2011,635477,71986.68
2008,150000,57443.26
2012,769084,51443.83
2015,231140,38018.51
2006,181223,36004.99
2009,468282,32293.75
2007,427143,31798.45
2003,521747,26504.06
1975,9083,23888.14
1995,134647,23508.07


Top 15 selling books in 2011 by sales rank

In [26]:
%%sql
SELECT * 
FROM book_sales
WHERE Publishing_Year = 2011 AND book_name != 'None'
ORDER BY sales_rank
ASC
LIMIT 15;

 * sqlite:///C:\Users\esait\book_db.db
Done.


index,Publishing_Year,Book_Name,Author,language_code,Author_Rating,Book_average_rating,Book_ratings_count,genre,gross_sales,publisher_revenue,sale_price,sales_rank,Publisher,units_sold
4,2011.0,Daughter of Smoke & Bone,Laini Taylor,eng,Intermediate,4.04,198283,fiction,37952.5,22771.5,7.99,4,Penguin Group (USA) LLC,4750
6,2011.0,The Power of Habit,Charles Duhigg,eng,Intermediate,4.03,155977,fiction,27491.67,16495.002,6.99,6,HarperCollins Publishers,3933
36,2011.0,The Language of Flowers,Vanessa Diffenbaugh,eng,Intermediate,4.07,140076,fiction,1964.16,1178.496,0.99,36,"Amazon Digital Services, Inc.",1984
52,2011.0,Silence,Becca Fitzpatrick,eng,Intermediate,4.16,190722,fiction,8517.93,0.0,4.99,54,"Amazon Digital Services, Inc.",1707
88,2011.0,"Thinking, Fast and Slow",Daniel Kahneman,eng,Intermediate,4.08,136950,fiction,1157.31,694.3860000000001,0.99,89,"Amazon Digital Services, Inc.",1169
101,2011.0,"Shatter Me (Shatter Me, #1)",Tahereh Mafi,en-US,Intermediate,4.01,176433,fiction,2195.2,1317.12,2.24,104,"Amazon Digital Services, Inc.",980
103,2011.0,I've Got Your Number,Sophie Kinsella,eng,Intermediate,3.93,144533,fiction,960.3,0.0,0.99,106,"Amazon Digital Services, Inc.",970
117,2011.0,Inheritance,Christopher Paolini,en-US,Intermediate,4.08,146905,nonfiction,10731.05,6438.63,11.99,121,Random House LLC,895
124,2011.0,Crossed,Ally Condie,eng,Intermediate,3.52,100886,fiction,841.5,504.9,0.99,130,"Amazon Digital Services, Inc.",850
166,2011.0,State of Wonder,Ann Patchett,eng,Intermediate,3.85,124837,fiction,1210.0,0.0,2.0,179,"Amazon Digital Services, Inc.",605


Total units sold, gross sales and publisher revenue by genre, ordered by gross sales

In [27]:
%%sql
SELECT genre, SUM(units_sold) AS 'Units Sold', ROUND(SUM(gross_sales), 2) AS 'Total Gross Sales', ROUND(SUM(publisher_revenue), 2) AS 'Total Publisher Revenue'
FROM book_sales
GROUP BY 1
ORDER BY 3
DESC;

 * sqlite:///C:\Users\esait\book_db.db
Done.


genre,Units Sold,Total Gross Sales,Total Publisher Revenue
fiction,8421492,1744525.46,762123.2
nonfiction,1634759,228158.87,131846.17
children,298118,13902.22,8341.33


Top 10 books by gross sales with total units sold and publisher revenue

In [28]:
%%sql
SELECT book_name, author, SUM(units_sold) AS 'Units Sold', CAST(Publishing_Year AS INT) AS 'Publishing_Year', Publisher, ROUND(SUM(gross_sales), 2) AS 'Total Gross Sales', ROUND(SUM(publisher_revenue), 2) AS 'Total Publisher Revenue'
FROM book_sales
WHERE book_name != 'None'
GROUP BY 1
ORDER BY 6
DESC
LIMIT 10;

 * sqlite:///C:\Users\esait\book_db.db
Done.


Book_Name,Author,Units Sold,Publishing_Year,Publisher,Total Gross Sales,Total Publisher Revenue
Go Set a Watchman,Harper Lee,5500,2015,"Amazon Digital Services, Inc.",47795.0,28677.0
When You Are Engulfed in Flames,David Sedaris,5500,2008,Hachette Book Group,41250.0,24750.0
Daughter of Smoke & Bone,Laini Taylor,4750,2011,Penguin Group (USA) LLC,37952.5,22771.5
Beowulf,"Unknown, Seamus Heaney",7000,1975,HarperCollins Publishers,34160.0,20496.0
The Power of Habit,Charles Duhigg,3933,2011,HarperCollins Publishers,27491.67,16495.0
The Virgin Suicides,Jeffrey Eugenides,2466,1993,Penguin Group (USA) LLC,26904.06,16142.44
Midnight in the Garden of Good and Evil,John Berendt,3800,1994,Hachette Book Group,26182.0,15709.2
Hopeless,Colleen Hoover,3733,2012,HarperCollins Publishers,26093.67,15656.2
A Little Princess,"Frances Hodgson Burnett, Nancy Bond",3666,1905,Random House LLC,23792.34,14275.4
The Velveteen Rabbit,"Margery Williams Bianco, William Nicholson",3333,1922,Random House LLC,21797.82,13078.69
